In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# FineTuning mT5 for summarization Task

For finetuning we have used the summarization tutorial given by hugging face:
https://huggingface.co/docs/transformers/en/tasks/summarization

In [ ]:
!pip install transformers datasets evaluate rouge_score

In [27]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
import pandas as pd
train=pd.read_csv("/kaggle/input/hindid/hindi_train.csv")

In [5]:
train.head()

,Id,Heading,Summary,Article
0,hindi_2023_train_0,"गला दबाकर हत्या की; बॉडी बोरे में भरी, लोकल मा...",Kerala Minor Girl Rape Case - केरल के एर्नाकुल...,केरल के एर्नाकुलम जिले में 5 साल की बच्ची से र...
1,hindi_2023_train_1,तेलंगाना में 18 की जान गई; जम्मू-कश्मीर में बा...,इस साल मानसून सीजन में कई राज्यों में भारी तबा...,मानसून सीजन में हुई भारी बारिश ने कई राज्यों म...
2,hindi_2023_train_2,"राजस्थान सरकार बनाएगी कर्ज राहत आयोग, कोर्ट के...",चुनावी साल में राजस्थान सरकार किसानों को लुभान...,चुनावी साल में राजस्थान सरकार किसानों को लुभान...
3,hindi_2023_train_3,"3 से 7 अगस्त तक कर सकेंगे अप्लाय, प्राइस बैंड ...",Non-banking lender SBFC Finance's initial publ...,नॉन बैंकिग फाइनेंस कंपनी 'SBFC फाइनेंस लिमिटेड...
4,hindi_2023_train_4,"डाइनिंग टेबल पर कुकर-कड़ाही न सजाएं, चीन के खा...",स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...,स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...


In [11]:
train=train.head(6000)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Id       6000 non-null   object
 1   Heading  6000 non-null   object
 2   Summary  6000 non-null   object
 3   Article  6000 non-null   object
dtypes: object(4)
memory usage: 187.6+ KB


In [4]:
train.duplicated().sum()

0

In [13]:
train

,Id,Heading,Summary,Article
0,hindi_2023_train_0,गला दबाकर हत्या की बॉडी बोरे में भरी लोकल मार्...,Kerala Minor Girl Rape Case - केरल के एर्नाकुल...,summarize: केरल के एर्नाकुलम जिले में साल की ब...
1,hindi_2023_train_1,तेलंगाना में की जान गई जम्मू-कश्मीर में बादल फ...,इस साल मानसून सीजन में कई राज्यों में भारी तबा...,summarize: मानसून सीजन में हुई भारी बारिश ने क...
2,hindi_2023_train_2,राजस्थान सरकार बनाएगी कर्ज राहत आयोग कोर्ट के ...,चुनावी साल में राजस्थान सरकार किसानों को लुभान...,summarize: चुनावी साल में राजस्थान सरकार किसान...
3,hindi_2023_train_3,से अगस्त तक कर सकेंगे अप्लाय प्राइस बैंड ₹ से...,Non-banking lender SBFC Finance's initial publ...,summarize: नॉन बैंकिग फाइनेंस कंपनी फाइनेंस लि...
4,hindi_2023_train_4,डाइनिंग टेबल पर कुकर-कड़ाही न सजाएं चीन के खान...,स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...,summarize: स्वाद खाने की बुनियाद है लेकिन अगर ...
...,...,...,...,...
5995,hindi_2023_train_5995,रिटायर्ड अफसर ने कहा- हमलावर हथियारों से लैस थ...,Conspiracy Behind London Indian High Commissio...,summarize: लंदन में भारतीय उच्चायोग पर मार्च क...
5996,hindi_2023_train_5996,कनाडाई प्रोफेसर ने में किया अटैक यह दूसरे विश्...,"Canadian Professor Hassan Diab Bombing Case; ""...",summarize: पेरिस में अक्टूबर को सिनेगॉग यहूदिय...
5997,hindi_2023_train_5997,इस पर सवार थे एक हजार ऑस्ट्रेलियाई कैदी में अम...,World War II Sunken Ship Wreckage Found In Sou...,summarize: ऑस्ट्रेलिया के इतिहास के सबसे बड़े ...
5998,hindi_2023_train_5998,रक्षा मंत्री बोले- युद्ध की आशंका के बीच चुनाव...,Pakistan's Defense Minister Raises Concerns of...,summarize: पाकिस्तान के रक्षा मंत्री ख्वाजा मु...


We have taken the preprocessing of the data from this link:
https://www.kaggle.com/code/endofnight17j03/hindi-text-short-summarization

In [12]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, ' ', data)

def preprocess_tokenize(text):
      # for removing punctuation from sentencesc
    text = str(text)
    text = re.sub(r'(\d+)', r'', text)
    
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = text.replace('\t', ' ')
    text = text.replace('\u200d', '')
    text=re.sub("(__+)", ' ', str(text)).lower()   #remove _ if it occors more than one time consecutively
    text=re.sub("(--+)", ' ', str(text)).lower()   #remove - if it occors more than one time consecutively
    text=re.sub("(~~+)", ' ', str(text)).lower()   #remove ~ if it occors more than one time consecutively
    text=re.sub("(\+\++)", ' ', str(text)).lower()   #remove + if it occors more than one time consecutively
    text=re.sub("(\.\.+)", ' ', str(text)).lower()   #remove . if it occors more than one time consecutively
    text=re.sub(r"[<>()|&©@#ø\[\]\'\",;:?.~*!]", ' ', str(text)).lower() #remove <>()|&©ø"',;?~*!
    text = re.sub(r"[‘’।:]", " ", str(text)) #removing other special characters
    text = re.sub("([a-zA-Z])",' ',str(text)).lower()
    text = re.sub("(\s+)",' ',str(text)).lower()
    text = remove_emojis(text)
    return text

train["Heading"]=train["Heading"].apply(preprocess_tokenize)

train["Article"]=train["Article"].apply(lambda x: "summarize: " + preprocess_tokenize(x))

In [14]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train)
train_dataset = train_dataset.train_test_split(test_size=0.1)
train_dataset

DatasetDict({
    train: Dataset({
        features: ['Id', 'Heading', 'Summary', 'Article'],
        num_rows: 5400
    })
    test: Dataset({
        features: ['Id', 'Heading', 'Summary', 'Article'],
        num_rows: 600
    })
})

In [11]:
train_dataset

Dataset({
    features: ['Id', 'Heading', 'Summary', 'Article'],
    num_rows: 1000
})

DatasetDict({
    train: Dataset({
        features: ['Id', 'Heading', 'Summary', 'Article'],
        num_rows: 900
    })
    test: Dataset({
        features: ['Id', 'Heading', 'Summary', 'Article'],
        num_rows: 100
    })
})

In [15]:
from transformers import AutoTokenizer

checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [16]:
tokenizer

T5TokenizerFast(name_or_path='google/mt5-small', vocab_size=250100, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [18]:
prefix = ""


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["Summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [19]:
tokenized_dataset_train= train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

2024-04-25 12:46:24.317091: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 12:46:24.317212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 12:46:24.456151: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [20]:
data_collator

DataCollatorForSeq2Seq(tokenizer=T5TokenizerFast(name_or_path='google/mt5-small', vocab_size=250100, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, model='google/mt5-small', padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100, return_tensors='pt')

In [ ]:
!pip install evaluate

In [ ]:
!pip install rouge_score

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [21]:
import numpy as np


def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=False)

#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
#     result["gen_len"] = np.mean(prediction_lens)
    return {}
    return {k: round(v, 4) for k, v in result.items()}

In [22]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [23]:
model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [24]:
import torch

# Clear CUDA cache
torch.cuda.empty_cache()

In [28]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/finetuned-mt5small",
    do_train=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    logging_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train["train"],
    eval_dataset=tokenized_dataset_train["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ·····································


wandb: ERROR API key must be 40 characters long, yours was 37
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 1177, in init
    wi.setup(kwargs)
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 301, in setup
    wandb_login._login(
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_login.py", line 334, in _login
    wlogin.prompt_api_key()
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_login.py", line 256, in prompt_api_key
    key, status = self._prompt_api_key()
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_login.py", line 236, in _prompt_api_key
    key = apikey.prompt_api_key(


Error: An unexpected error occurred

In [10]:
text = '''summarize:

विचारों को ढोते नहीं निचोड़ते थे राहुल
राहुल सांकृत्यायन के जीवन और उनकी लेखनी को पढ़ने समझने के बाद हम बहुत हद तक इस निष्कर्ष पर पहुंचते हैं कि विचारों के भवसागर में फंसने में बहुत अधिक विश्वास नहीं करते थे. उनके अंदर गजब की बेचैनी थी. संस्कृत की पढ़ाई के बाद आर्यसमाज के साथ जुड़े, जितना कुछ ज्ञान उन्हें आर्यसमाज से लेना था उसके बाद उनका आकर्षण बौद्ध धर्म के प्रति हो गया. भारत और तिब्बत में बौद्ध धर्म की समृर्द्ध विरासत रही है.  बौद्ध साहित्य को उन्होंने इतना अधिक निचोड़ा कि आज भी बिहार और बंगाल के लाइब्रेरी उसकी बदौलत सम्मानित है. राहुल जी के अनुसार जो विचार पुराने पड़ जाएं उसे छोड़ते हुए आगे बढ़ते जाना चाहिए. .'''

In [12]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/mymodel-finetuned-mt5/finetuned-mt5small")
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/input/mymodel-finetuned-mt5/finetuned-mt5small")



You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:

inputs = tokenizer(text, return_tensors="pt").input_ids

outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'उत्तरों को ढोते नहीं निचोड़ते थे राहुल राहुल सांकृत्यायन के जीवन और उनकी लेखनी को पढ़ने समझने के बाद हम बहुत हद तक इस निष्कर्ष पर पहुंचते हैं कि विचारों के भवसागर में फंसने में बहुत अधिक विश्वास नहीं करते थे'

In [29]:

inputs = tokenizer("summarize: " + train_dataset["test"][0]["Article"], return_tensors="pt").input_ids
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)
tokenizer.decode(outputs[0], skip_special_tokens=True)


'उत्तर प्रदेश की सुक्खू सरकार ने आपदा प्रभावितों को बड़ी राहत प्रदान की है आपदा प्रभावित क्षेत्रों का दौरा करके जमीनी हकीकत जानने के बाद शिमला लौटे मुख्यमंत्री ने प्रभावितों को विशेष राहत देने का फैसला लिया कुल्लू मंडी और सोलन में आपदा प्रभावितों को राहत प्रदान करने की घोषणा को अमली'

In [30]:
print(train_dataset["test"][0]["Summary"])

हिमाचल की सुक्खू सरकार ने आपदा प्रभावितों को बड़ी राहत प्रदान की है आपदा प्रभावित क्षेत्रों का दौरा करके जमीनी हकीकत जानने के बाद शिमला लौटे मुख्यमंत्री ने प्रभावितों को विशेष राहत देने का फैसला लिया 


In [ ]:
train["Summary"][3]

In [ ]:
import pandas as pd
test=pd.read_csv("/kaggle/input/hindid/HindiNews_test.csv")

In [ ]:
test.head()

In [ ]:
test["Article"][0]

In [ ]:
test["Heading"][0]

In [ ]:
sent=test["Article"][0]
t=f"summarize: {sent}"
inputs = tokenizer(t, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)
tokenizer.decode(outputs[0], skip_special_tokens=True)


In [9]:
model.save_pretrained("/kaggle/working/ft_sm", from_pt=True) 

# PreTrained Knowledge Distillation mT5

Using This paper as a precursor for distiling T5. by reducing decoder layers in student model we hope to achieve a relatively shorter model with faster inference and shorter training time. Choosing cleverly we may be able to retain the orignal performance

We have taken the preprocessing of the data from this link:
https://www.kaggle.com/code/endofnight17j03/hindi-text-short-summarization

In [2]:
import pandas as pd
train=pd.read_csv("/kaggle/input/hindid/hindi_train.csv")
train=train.head(10000)

import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, ' ', data)

def preprocess_tokenize(text):
      # for removing punctuation from sentencesc
    text = str(text)
    text = re.sub(r'(\d+)', r'', text)
    
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = text.replace('\t', ' ')
    text = text.replace('\u200d', '')
    text=re.sub("(__+)", ' ', str(text)).lower()   #remove _ if it occors more than one time consecutively
    text=re.sub("(--+)", ' ', str(text)).lower()   #remove - if it occors more than one time consecutively
    text=re.sub("(~~+)", ' ', str(text)).lower()   #remove ~ if it occors more than one time consecutively
    text=re.sub("(\+\++)", ' ', str(text)).lower()   #remove + if it occors more than one time consecutively
    text=re.sub("(\.\.+)", ' ', str(text)).lower()   #remove . if it occors more than one time consecutively
    text=re.sub(r"[<>()|&©@#ø\[\]\'\",;:?.~*!]", ' ', str(text)).lower() #remove <>()|&©ø"',;?~*!
    text = re.sub(r"[‘’।:]", " ", str(text)) #removing other special characters
    text = re.sub("([a-zA-Z])",' ',str(text)).lower()
    text = re.sub("(\s+)",' ',str(text)).lower()
    text = remove_emojis(text)
    return text

train["Summary"]=train["Summary"].apply(lambda x : preprocess_tokenize(x))

train["Article"]=train["Article"].apply(lambda x : "summarize: " + preprocess_tokenize(x))


from datasets import Dataset
train_dataset = Dataset.from_pandas(train)
train_dataset = train_dataset.train_test_split(test_size=0.1)
train_dataset

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/mymodel-finetuned-mt5/finetuned-mt5small")
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/input/mymodel-finetuned-mt5/finetuned-mt5small")


/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
train_dataset["train"][0]

{'Id': 'hindi_2023_train_7015',
 'Heading': 'मनोज बाजपेयी बोले- मैं ब्राह्मण परिवार से था..फिर भी मुस्लिम से शादी करने पर फैमिली को नहीं थी आपत्ति',
 'Summary': ' - - मनोज बाजपेयी ने हाल ही में अपनी इंटरफेथ मैरिज को लेकर बात की उन्होंने कहा कि वो कभी भी अपने घर पर रिलीजन को लेकर कोई चर्चा नहीं करते हैं जब उन्होंने शादी करने का फैसला किया तो उन्हें अपनी फैमिली से',
 'Article': 'summarize: मनोज बाजपेयी ने हाल ही में अपनी इंटरफेथ मैरिज को लेकर बात की उन्होंने कहा कि वो कभी भी अपने घर पर रिलीजन को लेकर कोई चर्चा नहीं करते हैं जब उन्होंने शादी करने का फैसला किया तो उन्हें अपनी फैमिली से किसी तरह से विरोध का सामना नहीं करना पड़ा था मनोज के मुताबिक वो एक प्राउड हिंदू हैं जबकि उनकी वाइफ शबाना भी एक प्राउड मुस्लिम हैं दोनों के रिलेशनशिप में कभी धर्म आड़े नहीं आया मनोज का कहना है कि वो और शबाना एक दूसरे के धार्मिक भावनाओं का काफी सम्मान करते हैं मैं ब्रह्माण परिवार से था लेकिन किसी को नहीं थी आपत्ति बरखा दत्त को दिए एक इंटरव्यू में मनोज ने कहा वैल्यूज की वजह से हमारे बीच रिश्ता चल रहा है अगर हम 

In [4]:
import copy
m2 = copy.deepcopy(model)

In [5]:
for i in range(3):
    m2.decoder.get_submodule("block").pop(1)
for i in range(2):
    m2.decoder.get_submodule("block").pop(2)

In [6]:

import torch
def hidden_state_loss():
    phi = {
        0 : 3,
        1 : 6,
        2 : 7
    }
    loss = 0
    for i in phi.keys():
        param = m2.decoder.get_submodule("block")[i]
        param = param.state_dict()
        
        ppar = model.decoder.get_submodule("block")[phi[i]]
        ppar = ppar.state_dict()
        for name, param1 in param.items():
            if name in ppar.keys():
                param2 = ppar[name]
                loss += torch.sqrt(torch.sum(torch.square(param2 - param1)) / sum(param1.size()))
        del param, ppar
    return loss / 3

hidden_state_loss()

tensor(83.2906)

In [ ]:
ppar = model.decoder.get_submodule("block").state_dict().keys()

In [91]:
model.decoder.get_submodule("block")[1].state_dict().keys()

odict_keys(['layer.0.SelfAttention.q.weight', 'layer.0.SelfAttention.k.weight', 'layer.0.SelfAttention.v.weight', 'layer.0.SelfAttention.o.weight', 'layer.0.layer_norm.weight', 'layer.1.EncDecAttention.q.weight', 'layer.1.EncDecAttention.k.weight', 'layer.1.EncDecAttention.v.weight', 'layer.1.EncDecAttention.o.weight', 'layer.1.layer_norm.weight', 'layer.2.DenseReluDense.wi_0.weight', 'layer.2.DenseReluDense.wi_1.weight', 'layer.2.DenseReluDense.wo.weight', 'layer.2.layer_norm.weight'])

In [61]:
m2.save_pretrained("/kaggle/working/student",from_pt=True)

In [66]:
sample_toks = tokenizer(text=train_dataset["train"][0]["Article"], text_target=train_dataset["train"][0]["Summary"],padding = "max_length", max_length=128, truncation=True,return_tensors="pt",return_attention_mask=True)


In [68]:
import torch
m_o = model(sample_toks["input_ids"], attention_mask=sample_toks["attention_mask"],labels=sample_toks["labels"])
m_o2 = m2(sample_toks["input_ids"], attention_mask=sample_toks["attention_mask"],labels=torch.argmax(m_o2.logits, dim=-1))

In [69]:
torch.argmax(m_o2.logits, dim=-1)

tensor([[   259,    259,    259,    259,    259,    259,    259,    259,    259,
            259,    259,    259,    259,    259,    259,    259,    259,    259,
            270, 214272,  45564,  92075,  79409, 235206, 109896, 231096,  47365,
          58986,  99212,  99212,   9824,  73620, 111577,  46658, 184702,  49213,
          66165,  96613, 218943,   1691,  77723,  32290, 171480, 208287,  94239,
         242711, 104310,  19426,    211, 115086, 150510,    259,    259, 214732,
         101340,    259,    259, 145421, 155383,      1, 141360, 244447, 248723,
         233053, 185992,  46224,  55042, 193507, 175964, 200363,    259, 211932,
         151557, 178047,    259, 230647, 109055, 184283, 109777, 135897, 162071,
         151084,   9989, 199685, 188060, 123932,    259, 151692,  36961,    273,
         124900, 124900,  55115, 242399,    259, 237824,  73620,    259,    259,
          90320,  91950, 232439, 205598, 151643, 151643, 151643, 151643, 151643,
         151643, 117878,  91

In [67]:
sample_toks.labels

tensor([[  2139,  17259,   1396,    757, 129884,   6759,   2297,   6491,  96463,
          54990,  51750,  31466,  10022,   5362,   3689,   7141,   1304,    259,
          33236,  63539,   1100,    757,   4315,  29929,    641,  34973,   1945,
           3927,   7141,   1162,  52059,    996,  41667,    975,   5252,  11221,
           1100,   1484,   2948,   1871,    259,  47102,   3638,   1114,   2075,
            830,      1,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,    

In [48]:
tokenizer.decode(torch.argmax(m_o2.logits, dim=-1)[0])

'               әтរិតCheckRates在内的ښت必ずεμα গ্র稱\U000f0004   nithe諤edlyણાদ্বøkke Faовини FarendeltоодealedpreventDefault </s> ऑগ্রікealedmistaпрямо udad間違いない  nquire</s>dgive密集abhra\ue63aauntաբանությունadıntávनाइинч可在entsiaalanntエンドミルoterapeutanntυτερ</s> इतachment</s>таво̍электрическzzati命运аҳм <0x3F>ැහැange</s></s>:#333реат atolicaaridabhrabhrbrydebrydebrydebrydebrydebrydebrydebrydebrydebrydeuavකරුවන්terreertificadoens贯彻落实涕</s>्रास kuts寨dahkanしているのは\U000f0004ने장관 ນິຍົມurisーニ'

In [72]:
import torch.nn.functional as F
F.cross_entropy(m_o2.logits.view(-1, m_o2.logits.size(-1)),torch.argmax(m_o.logits, dim=-1).view(-1)) #.view(-1, m_o.logits.size(-1))

tensor(29.3915, grad_fn=<NllLossBackward0>)

In [73]:
torch.cuda.empty_cache()

In [7]:
from torch.utils.data import Dataset, DataLoader
train_loader = DataLoader(train_dataset["train"], batch_size=4, shuffle=True)
val_loader = DataLoader(train_dataset["test"], batch_size=4, shuffle=True)

fine tuning 

In [34]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
# Custom loss function
def custom_loss(out_norm, out_small, labels):
    g_lables = F.cross_entropy(out_norm.logits.view(-1, out_norm.logits.size(-1)),labels.view(-1)) 
    return g_lables  

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# m2.to(device)
model.to(device)
# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=5e-4)
def train_loop(data_loader):
#     m2.train()
    model.train()
    total_loss = 0
    for i in tqdm(data_loader):
        tokens = tokenizer(text=i["Article"], text_target=i["Heading"],padding = "max_length", max_length=128, truncation=True,return_tensors="pt",return_attention_mask=True)
        input_ids = tokens['input_ids'].to(device)
        attention_mask = tokens['attention_mask'].to(device)
        labels = tokens['labels'].to(device)

        optimizer.zero_grad()
        
        outputs_teacher = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
     
        loss = custom_loss(outputs_teacher, outputs_teacher, labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    return total_loss
    
def val_loop(data_loader):
#     m2.eval()
    model.eval()
    total_loss = 0
    for i in tqdm(data_loader):
        tokens = tokenizer(text=i["Article"], text_target=i["Heading"],padding = "max_length", max_length=128, truncation=True,return_tensors="pt",return_attention_mask=True)
        input_ids = tokens['input_ids'].to(device)
        attention_mask = tokens['attention_mask'].to(device)
        labels = tokens['labels'].to(device)

        with torch.no_grad():
            outputs_teacher = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
        
            loss = custom_loss(outputs_teacher,outputs_teacher, labels)
            total_loss += loss.item()

    return total_loss
# Training loop
num_epochs = 10
loss_training = []
loss_val = []
for epoch in range(num_epochs):
    loss_training.append(train_loop(train_loader))
    loss_val.append(val_loop(val_loader))
    if(loss_val[-1] == min(loss_val)):
        model.save_pretrained("/kaggle/working/teacher_best",from_pt=True)
    print(f'Epoch {epoch+1}/{num_epochs}, Trn Loss: {loss_training[-1]/len(train_loader)},Val Loss: {loss_val[-1]/len(val_loader)}')
    
    inputs = tokenizer( train_dataset["test"][1]["Article"], return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(inputs, max_new_tokens=128, do_sample=False)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    print(train_dataset["test"][1]["Heading"])

print("Training completed!")


100%|██████████| 150/150 [00:09<00:00, 15.18it/s]


Epoch 1/10, Trn Loss: nan,Val Loss: nan

जगह - जगह बारिश बाढ़ और पुलों की जल समाधि के मायने


 20%|██        | 270/1350 [00:52<03:29,  5.16it/s]


KeyboardInterrupt: 

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
# Custom loss function
def custom_loss(out_norm, out_small, labels):
    g_lables = F.cross_entropy(out_norm.logits.view(-1, out_norm.logits.size(-1)),torch.argmax(out_small.logits, dim=-1).view(-1)) 
#     logits =  torch.sqrt((F.mse_loss(
#         torch.clamp(F.log_softmax(out_norm.logits, dim=-1),1e-8,1-1e-8),
#         torch.clamp(F.log_softmax(out_small.logits, dim=-1),1e-8,1-1e-8),
#         )))
#     hidden = hidden_state_loss()
#     print(g_lables, logits, 3*hidden) 
    return g_lables  #+ 0.8*hidden logits +

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
m2.to(device)
model.to(device)
# Optimizer
optimizer = optim.AdamW(m2.parameters(), lr=5e-4)
def train_loop(data_loader):
    m2.train()
    model.eval()
    total_loss = 0
    for i in tqdm(data_loader):
        tokens = tokenizer(text=i["Article"], text_target=i["Summary"],padding = "max_length", max_length=128, truncation=True,return_tensors="pt",return_attention_mask=True)
        input_ids = tokens['input_ids'].to(device)
        attention_mask = tokens['attention_mask'].to(device)
        labels = tokens['labels'].to(device)

        optimizer.zero_grad()
        
        outputs_teacher = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
        
        outputs_student = m2(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels= torch.argmax(outputs_teacher.logits,-1))
        
        
        loss = custom_loss(outputs_student,outputs_teacher, labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    return total_loss
    
def val_loop(data_loader):
    m2.eval()
    model.eval()
    total_loss = 0
    for i in tqdm(data_loader):
        tokens = tokenizer(text=i["Article"], text_target=i["Summary"],padding = "max_length", max_length=128, truncation=True,return_tensors="pt",return_attention_mask=True)
        input_ids = tokens['input_ids'].to(device)
        attention_mask = tokens['attention_mask'].to(device)
        labels = tokens['labels'].to(device)

#         optimizer.zero_grad()
        with torch.no_grad():
            outputs_teacher = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
            outputs_student = m2(input_ids=input_ids,
                            attention_mask=attention_mask,
                            labels=torch.argmax(outputs_teacher.logits,-1))
            
        
            loss = custom_loss(outputs_student,outputs_teacher, labels)
            total_loss += loss.item()

    return total_loss
# Training loop
num_epochs = 10
loss_training = []
loss_val = []
for epoch in range(num_epochs):
    loss_training.append(train_loop(train_loader))
    loss_val.append(val_loop(val_loader))
    if(loss_val[-1] == min(loss_val)):
        m2.save_pretrained("/kaggle/working/student_best",from_pt=True)
    print(f'Epoch {epoch+1}/{num_epochs}, Trn Loss: {loss_training[-1]/len(train_loader)},Val Loss: {loss_val[-1]/len(val_loader)}')
    
    inputs = tokenizer( train_dataset["test"][1]["Article"], return_tensors="pt").input_ids.to("cuda")
    outputs = m2.generate(inputs, max_new_tokens=128, do_sample=False)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    print(train_dataset["test"][1]["Summary"])

print("Training completed!")


100%|██████████| 250/250 [00:22<00:00, 11.23it/s]


Epoch 1/10, Trn Loss: 2.156845547702577,Val Loss: 1.0614953959584237
उत्तरभाजपा सांसद और केंद्रीय कानून और न्याय राज्य मंत्री प्रो सत्यपाल सिंह बघेल ने कहा कि उनकी संख्या हजारों में भी नहीं है ये लोग उप राष्ट्रपति बनने के लिए सहिष्णु होने का मुखौटा पहन लेते हुए थे                                                          
 भाजपा सांसद और केंद्रीय मंत्री सत्यपाल सिंह बघेल ने सोमवार को कहा कि देश में सहिष्णु मुसलमानों को उंगलियों पर गिना जा सकता है मुझे लगता है कि उनकी संख्या हजारों में भी नहीं है 


100%|██████████| 250/250 [00:22<00:00, 11.23it/s]


Epoch 2/10, Trn Loss: 1.1098835388024648,Val Loss: 0.8682453416883945
उत्तरभाजपा सांसद और केंद्रीय कानून और न्याय राज्य मंत्री प्रो सत्यपाल सिंह बघेल ने कहा कि देश में सहिष्णु मुसलमानों को उंगलियों पर गिना जा सकता है मुझे लगता है कि उनकी संख्या हजारों में भी नहीं है ये लोग उप राष्ट्रपति बनेंगे
 भाजपा सांसद और केंद्रीय मंत्री सत्यपाल सिंह बघेल ने सोमवार को कहा कि देश में सहिष्णु मुसलमानों को उंगलियों पर गिना जा सकता है मुझे लगता है कि उनकी संख्या हजारों में भी नहीं है 


100%|██████████| 250/250 [00:21<00:00, 11.46it/s]


...

Epoch 4/10, Trn Loss: 0.8649334011938837,Val Loss: 0.8261885535418987
उत्तरभाजपा सांसद और केंद्रीय कानून और न्याय राज्य मंत्री प्रो सत्यपाल सिंह बघेल ने कहा कि देश में सहिष्णु मुसलमानों को उंगलियों पर गिना जा सकता है मुझे लगता है कि उनकी संख्या हजारों में हम किस तरह से बच गए थे
 भाजपा सांसद और केंद्रीय मंत्री सत्यपाल सिंह बघेल ने सोमवार को कहा कि देश में सहिष्णु मुसलमानों को उंगलियों पर गिना जा सकता है मुझे लगता है कि उनकी संख्या हजारों में भी नहीं है 


100%|██████████| 250/250 [00:21<00:00, 11.41it/s]


Epoch 5/10, Trn Loss: 0.8151134886708524,Val Loss: 0.8084485946595669
उत्तरभाज भाजपा सांसद और केंद्रीय कानून और न्याय राज्य मंत्री प्रो सत्यपाल सिंह बघेल ने कहा कि देश में सहिष्णु मुसलमानों को साथ लेकर चलना चाहिए
 भाजपा सांसद और केंद्रीय मंत्री सत्यपाल सिंह बघेल ने सोमवार को कहा कि देश में सहिष्णु मुसलमानों को उंगलियों पर गिना जा सकता है मुझे लगता है कि उनकी संख्या हजारों में भी नहीं है 


  2%|▏         | 40/2250 [00:07<07:08,  5.16it/s]


KeyboardInterrupt: 

In [11]:

inputs = tokenizer( train_dataset["test"][70]["Article"], return_tensors="pt").input_ids.to("cuda")
outputs = m2.generate(inputs, max_new_tokens=128, do_sample=False)
tokenizer.decode(outputs[0], skip_special_tokens=True)


'उत्तर- थाहां और क्या है कसीनो और कि क्या है कसीनो जऔर क्यासे लोग इस इसऔर                                                                                                    '

In [12]:
print(train_dataset["test"][70]["Summary"])

 - कसीनो लीगली जुआं खेलने की एक जगह होती है जहां कुछ लोग रातों-रात करोड़पति बनते हैं तो किसी के हाथ कुछ नहीं लगता


## Distil mT5 loading

In [13]:
x2 = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/input/distil-mt5/student_best").to("cuda")

Some weights of MT5ForConditionalGeneration were not initialized from the model checkpoint at /kaggle/input/distil-mt5/student_best and are newly initialized: ['decoder.block.3.layer.0.SelfAttention.k.weight', 'decoder.block.3.layer.0.SelfAttention.o.weight', 'decoder.block.3.layer.0.SelfAttention.q.weight', 'decoder.block.3.layer.0.SelfAttention.v.weight', 'decoder.block.3.layer.0.layer_norm.weight', 'decoder.block.3.layer.1.EncDecAttention.k.weight', 'decoder.block.3.layer.1.EncDecAttention.o.weight', 'decoder.block.3.layer.1.EncDecAttention.q.weight', 'decoder.block.3.layer.1.EncDecAttention.v.weight', 'decoder.block.3.layer.1.layer_norm.weight', 'decoder.block.3.layer.2.DenseReluDense.wi_0.weight', 'decoder.block.3.layer.2.DenseReluDense.wi_1.weight', 'decoder.block.3.layer.2.DenseReluDense.wo.weight', 'decoder.block.3.layer.2.layer_norm.weight', 'decoder.block.4.layer.0.SelfAttention.k.weight', 'decoder.block.4.layer.0.SelfAttention.o.weight', 'decoder.block.4.layer.0.SelfAttentio

In [19]:
model.to("cuda")
inputs = tokenizer(train_dataset["test"][16]["Article"], return_tensors="pt").input_ids.to("cuda")
outputs = x2.generate(inputs, max_new_tokens=128, do_sample=False)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print("___________________________________________________")
print(train_dataset["test"][16]["Summary"])
print("___________________________________________________")
print(train_dataset["test"][16]["Article"])

उत्तरनौकरी की तैयारी में जुटे युवाओं के लिए नौकरी के मौके आए हैं पहली नौकरी इंटीग्रल कोच फैक्ट्री में अप्रेंटिस के पदों पर है साल के युवा इस नौकरी के लिए अप्लाई कर सकते हैं दूसरी भर्ती रायपुर में सीनियर रेजिडेंट के पदों पर है                                     
___________________________________________________
नौकरी की तैयारी में जुटे युवाओं के लिए नौकरी के मौके आए हैं पहली नौकरी इंटीग्रल कोच फैक्ट्री में अप्रेंटिस के पदों पर है साल के युवा इस नौकरी के लिए अप्लाई कर सकते हैं दूसरी भर्ती रायपुर में सीनियर रेजिडेंट 
___________________________________________________
summarize: नौकरी की तैयारी में जुटे युवाओं के लिए नौकरी के मौके आए हैं पहली नौकरी इंटीग्रल कोच फैक्ट्री में अप्रेंटिस के पदों पर है साल के युवा इस नौकरी के लिए अप्लाई कर सकते हैं दूसरी भर्ती रायपुर में सीनियर रेजिडेंट के पदों पर है / कर चुके हैं तो इसके लिए अप्लाई कर सकते हैं इस नौकरी के लिए जुलाई तक अप्लाई कर सकते हैं वहीं साल के अभ्यर्थियों के लिए सेंटर फॉर डेवलपमेंट ऑफ टेलीमेटिक्स में पदों पर वैकेंसी है साथ ही साल के य

In [20]:
articles = train["Article"]
summaries = train["Summary"]

In [33]:
SystemSummary1 = []
labels = []
for i in tqdm(val_loader):
    x2.eval()
    inputs = tokenizer(text=i["Article"], text_target=i["Summary"],padding = "max_length", max_length=128, truncation=True,return_tensors="pt",return_attention_mask=True)
    inputs = inputs.input_ids.to("cuda")
    outputs = x2.generate(inputs, max_new_tokens=128, do_sample=False)
    SystemSummary1.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
    labels.append(i["Summary"][0])

100%|██████████| 250/250 [07:29<00:00,  1.80s/it]


In [27]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [35]:
from rouge import Rouge

df = pd.DataFrame({"summary": labels, "s_sum": SystemSummary1})
# print(==df.head(3))
rouge = Rouge()
score1 = rouge.get_scores(df['s_sum'], df['summary'], avg=True)
BartRouge1 = pd.DataFrame(score1).set_index([['recall','precision','f-measure']])
BartRouge1

,rouge-1,rouge-2,rouge-l
recall,0.688883,0.591462,0.656319
precision,0.525687,0.436652,0.503156
f-measure,0.578575,0.483363,0.552998


# CrossLingual KD of mT5 (the translation model can prefferably be changed)